In [1]:
import json

all_edits = json.loads(open('result.json', 'r', encoding='utf8').read())

In [6]:
def search(edit_type_q=None, head_q={}, target_q={}, child_q={}, delete_q={}):
    groups = filter(lambda e: e['edit_type'] == edit_type_q, all_edits) if edit_type_q else all_edits
    
    def extract(group, node):
        return map(lambda e: e[node], group['edits']) 
        
    def match(nodes, query):
        for node in nodes:
            if not node: continue
            
            correct = sum([query[key] == node[key] for key in query])
            if correct == len(query):
                return True
        return False
    
    if head_q:
        groups = filter(lambda g: match(extract(g, 'Head'), head_q), groups)
    
    if target_q:
        groups = filter(lambda g: match(extract(g, 'Target'), target_q), groups)
    
    if child_q:
        get_childs = lambda g: [c for child in extract(g, 'Child') for c in child] # Extract and flatten
        groups = filter(lambda g: match(get_childs(g), child_q), groups)
    
    if delete_q:
        get_deletes = lambda g: [d for delete in extract(g, 'Delete') for d in delete] # Extract and flatten
        groups = filter(lambda g: match(get_deletes(g), delete_q), groups)

    return groups

def output(groups):
    def format_line(node):
        return node['token'] + "\t" + node['lemma'] + "\t" + node['tag'] + "\t" + \
            (node['dep'] if 'dep' in node and node['dep'] else '')
        
    for group in groups:
        print("==========================")
        print("type:", group['edit_type'])
        print("sent:", group['sent'])
        print()
        
        for edit in group['edits']:
            head, target, childs, deletes = edit['Head'], edit['Target'], edit['Child'], edit['Delete']
            
            if head: print("Head:\t" + format_line(head))
            if target: print("Target:\t" + format_line(target))
                
            for child in childs:
                print("Child:\t" + format_line(child))
            
            print()
            for delete in deletes:
                print("Delete:\t" + format_line(delete))
            print()

In [7]:
groups = search(edit_type_q='', target_q={'token': 'argue'})
output(groups)

type: Replace
sent: We [-tend　sometimes　to　argue>>tend　to　argue　sometimes+} but we are willing to discover new games together .

Head:	tend	tend	VBP	
Target:	tend	tend	VBP	ROOT
Child:	We	-PRON-	PRP	nsubj
Child:	argue	argue	VB	xcomp
Child:	but	but	CC	cc
Child:	are	be	VBP	conj

Delete:	tend	tend	VB	
Delete:	sometimes	sometimes	RB	
Delete:	to	to	TO	
Delete:	argue	argue	VB	

Head:	argue	argue	VB	
Target:	to	to	TO	aux

Delete:	tend	tend	VB	
Delete:	sometimes	sometimes	RB	
Delete:	to	to	TO	
Delete:	argue	argue	VB	

Head:	tend	tend	VBP	
Target:	argue	argue	VB	xcomp
Child:	to	to	TO	aux
Child:	sometimes	sometimes	RB	advmod

Delete:	tend	tend	VB	
Delete:	sometimes	sometimes	RB	
Delete:	to	to	TO	
Delete:	argue	argue	VB	

Head:	argue	argue	VB	
Target:	sometimes	sometimes	RB	advmod

Delete:	tend	tend	VB	
Delete:	sometimes	sometimes	RB	
Delete:	to	to	TO	
Delete:	argue	argue	VB	

type: Replace
sent: We love the same things end each one usually kid the other so it 's very difficult for us [-arguing　.>

Child:	.	.	.	punct

Delete:	argues	argue	VBZ	

type: Replace
sent: We both are teachers and often [-arguing>>argue+} about our work .

Head:	are	be	VBP	
Target:	argue	argue	VBP	conj
Child:	often	often	RB	advmod
Child:	about	about	IN	prep

Delete:	arguing	argue	VBG	

type: Replace
sent: Well , they pretended that they did n't know each other , but one day , they started [-arguing>>to　argue+} and fight .

Head:	argue	argue	VB	
Target:	to	to	TO	aux

Delete:	arguing	argue	VBG	

Head:	started	start	VBD	
Target:	argue	argue	VB	xcomp
Child:	to	to	TO	aux
Child:	and	and	CC	cc
Child:	fight	fight	VB	conj

Delete:	arguing	argue	VBG	

type: Delete
sent: They have lost out on state projects and some of them argue [-a-] criminal bribe -taking .

Head:	lost	lose	VBN	
Target:	argue	argue	VBP	conj
Child:	some	some	DT	nsubj
Child:	-taking	-taking	NN	ccomp
Child:	.	.	.	punct

Delete:	a	a	DT	

Head:	bribe	bribe	NN	
Target:	criminal	criminal	JJ	amod

Delete:	a	a	DT	

type: Replace
sent: The COO disagreed wi